# Validation Set Approach

In [63]:
library(ISLR)
set.seed(1)
train=sample(392,196) #select a random subset of 192 obs out of 396 obs

In [64]:
lm=lm(mpg~horsepower,data=Auto,subset=train) #use subset option to fit only obs in train set
mean((mpg-predict(lm,Auto))[-train]^2)  #calulate MSE on test set
#predict(lm,Auto) takes the whole df and makes predictions just from the vars in the model
#(mpg-predict(lm,Auto))[-train] --> only errors NOT in the training data are selected

[1] 26.14142

In [71]:
lm2=lm(mpg~horsepower+I(horsepower^2),data=Auto,subset=train)
lm3=lm(mpg~poly(horsepower,2),data=Auto,subset=train)
mean((mpg-predict(lm2,Auto))[-train]^2)
mean((mpg-predict(lm3,Auto))[-train]^2)

[1] 19.82259

[1] 19.82259

# Leave-One-Out Cross-Validation

In [87]:
library(boot)
glm=glm(mpg~horsepower,data=Auto) #performs linear regression without the family argument
                         #we'll use this instead of lm because it has bootstrap abilities
cv.error=cv.glm(Auto,glm) #cv.glm() is part of the boot library
cv.error$delta #provides the LOOVC test error statistic in this case (both #s are the same)

[1] 24.23151 24.23114

In [104]:
cv.error=rep(0,5)
for (i in 1:5){
glm.fit=glm(mpg~poly(horsepower,i),data=Auto)
cv.error[i]=cv.glm(Auto,glm.fit)$delta[1]  
}    #each delta vector has 2 #s = LOOCV = the same, so we take the first => $delta[1]
cv.error

[1] 19.24821 19.24787

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

# K-Fold Cross Validation

In [110]:
set.seed(17)
glm=glm(mpg~poly(horsepower,2),data=Auto)
cv.error=cv.glm(Auto,glm,K=10)$delta[1] #set K=10 to do 10-fold CV
cv.error

[1] 19.27927

# The Bootstrap

In [122]:
library(boot)
alpha.fn=function(data,index){
X=data$X[index]
Y=data$Y[index]
return((var(Y)-cov(X,Y))/(var(X)+var(Y)-2*cov(X,Y))) } #create alpha statistic

set.seed(1)
alpha.fn(Portfolio ,1:100) #create alpha from first 100 obs
alpha.fn(Portfolio,sample(100,100,replace=T)) 
#create alpha from 100 randomly selected obs with replacement from the first 100 obs
boot(Portfolio ,alpha.fn,R=1000) 
# do the above operation to produce an estimated alpha 1000 times, but automated using boot()
#original alpha from all data  = .5758, and the bootstrap estimate for SE(est alpha) = .0886

[1] 0.5758321

[1] 0.5963833


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original        bias    std. error
t1* 0.5758321 -7.315422e-05  0.08861826

In [130]:
boot.fn=function(data,index)
return(coef(lm(mpg~horsepower,data=data,subset=index)))
    
boot.fn(Auto,1:392) #return coefficients
set.seed(1)
boot.fn(Auto,sample(392,392,replace=T)) #produce bootstrap estimate of coeffs
boot.fn(Auto,sample(392,392,replace=T)) #randomly sample with replacement from all 392 obs

(Intercept)  horsepower 
 39.9358610  -0.1578447

(Intercept)  horsepower 
 38.7387134  -0.1481952

(Intercept)  horsepower 
 40.0383086  -0.1596104

In [131]:
boot(Auto,boot.fn,1000) #produces bootstrap estimate of standard errors
                        #and the original coeffs from all data


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original      bias    std. error
t1* 39.9358610  0.02972191 0.860007896
t2* -0.1578447 -0.00030823 0.007404467

# R Lab 3

In [136]:
set.seed(3)
train=sample(392,196)
lm00=lm(mpg~horsepower,data=Auto,subset=train)
lm11=lm(mpg~poly(horsepower,2),data=Auto,subset=train)
lm22=lm(mpg~poly(horsepower,3),data=Auto,subset=train)

round(mean((mpg-predict(lm00,Auto))[-train]^2),2)
round(mean((mpg-predict(lm11,Auto))[-train]^2),2)
round(mean((mpg-predict(lm22,Auto))[-train]^2),2)

[1] 26.29

[1] 21.5

[1] 21.51

In [142]:
glm11=glm(mpg~poly(horsepower,6),data=Auto)
                        
cv.error=cv.glm(Auto,glm11) 
round(cv.error$delta[1],2)

[1] 18.98

In [154]:
set.seed(17)
glm111=glm(mpg~poly(horsepower ,1),data=Auto)
round(cv.glm(Auto,glm111,K=5)$delta[1][1],2)

glm222=glm(mpg~poly(horsepower ,2),data=Auto)
round(cv.glm(Auto,glm222,K=5)$delta[1][1],2)

[1] 24.26

[1] 19.15

In [150]:
set.seed(17)
cv.error.10=rep(0,10)
for (i in 1:10){
glm333=glm(mpg~poly(horsepower ,i),data=Auto)
cv.error.10[i]=cv.glm(Auto,glm333,K=5)$delta[1]
    }
cv.error.10

[1] 24.26240 19.15424 19.14205 19.42963 18.87288 19.36350 19.04407 18.91539
 [9] 19.22077 19.16847

In [157]:
set.seed(2)
boot(Portfolio ,alpha.fn,R=1000) 


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original      bias    std. error
t1* 0.5758321 0.001085673  0.09004091

In [159]:
round(0.09004091,3)

[1] 0.09

In [168]:
boot.fn=function(data,index)
coefficients(lm(mpg~horsepower+I(horsepower^2),data=data,
subset=index))
set.seed(2)
boot(Auto ,boot.fn ,1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  6.977646e-02 2.1445102521
t2* -0.466189630 -1.063154e-03 0.0337397614
t3*  0.001230536  4.078156e-06 0.0001210534

In [171]:
round(2.1445102521,4)
round(0.0337397614,4)
round(0.0001210534,4)

[1] 2.1445

[1] 0.0337

[1] 1e-04